In [ ]:
#Dependancies
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

#data read from CSV
columns = ['customer_no','create_dt','max_purchase_date','recency','frequency','revenue','subscriber']
df = pd.read_csv('/content/rmf_data.csv',names= columns)

#Scaling and standardisation
df['recency'][df['recency']< 0] = df['recency'].mean()
df.loc[df['frequency'] > 180, 'frequency'] = np.nan
df['frequency'] = df['frequency'].interpolate()
df.loc[df['revenue'] > 200000, 'revenue'] = np.nan
df['revenue'] = df['revenue'].interpolate()
df['revenue'] = df['revenue'].div(10)
mean_recency = df['recency'].mean()
df['recency'].fillna(mean_recency, inplace=True)

#Clustering
recency_data = df[['recency']]
kmeans = KMeans(n_clusters=3)
kmeans.fit(recency_data)
df['recency_cluster'] = kmeans.labels_

revenue_data = df[['revenue']]
kmeans = KMeans(n_clusters=3)
kmeans.fit(revenue_data)
df['revenue_cluster'] = kmeans.labels_

frequency_data = df[['frequency']]
kmeans = KMeans(n_clusters=3)
kmeans.fit(frequency_data)
df['frequency_cluster'] = kmeans.labels_

# Calculate the overall_score
df['overall_score'] = df['recency_cluster'] + df['frequency_cluster'] + df['revenue_cluster']

# Update 'overall_score' based on conditions
df.loc[(df['overall_score'] == 2) & (df['recency'] >= 400), 'overall_score'] = 1
df.loc[(df['overall_score'] == 3) & (df['recency'] >= 400), 'overall_score'] = 1
df.loc[(df['overall_score'] == 5) & (df['revenue'] >= 7100.00), 'overall_score'] = 6
df.loc[(df['overall_score'] == 1) & (df['recency'] >= 400) & (df['recency'] <= 1000), 'overall_score'] = 0
df.loc[(df['overall_score'] == 4) & (df['revenue'] >= 2000.00), 'overall_score'] = 5
df.loc[(df['overall_score'] == 5) & (df['recency'] >= 1000), 'overall_score'] = 1
df.loc[(df['overall_score'] == 5) & (df['recency'] >= 400) & (df['recency'] <= 1000), 'overall_score'] = 0


#Add lables
df.loc[df['overall_score']==0,'Segment'] = 'Slipping'
df.loc[df['overall_score']==1,'Segment'] = 'Lost Touch'
df.loc[df['overall_score']==2,'Segment'] = 'New Customer'
df.loc[df['overall_score']==3,'Segment'] = 'Faithful'
df.loc[df['overall_score']==4,'Segment'] = 'Loyal Purchaser'
df.loc[df['overall_score']==5,'Segment'] = 'Affluent'
df.loc[df['overall_score']==6,'Segment'] = 'Top Tier'

#CSV export
export_df = df[['customer_no','overall_score','Segment']]
export_df.to_csv('/content/MSO_RFM_SEGMENT.csv')